In [1]:
import numpy as np
from numpy.random import default_rng

from src.dlla.berg import make_mlp
from src.dlla.hw import prepare_traces_dl, dlla_known_p
from src.dlla.wegener import make_mlp_wegener, wegener_p
from src.trace_set.transform import reduce_fixed_fixed
from src.tvla.cri import tvla_t_test, rho_test
from src.tvla.tvla import prepare_tvla

rng = default_rng()

In [ ]:
def store_results(method: str, p: float):
    file_name = f"results_false_positives.csv"
    with open(file_name, 'a') as f:
        f.write(f"{method};{p}\n")


while True:
    num_profile = 10000
    num_attack = 2000
    num_sample_points = 1000

    x = np.array([rng.standard_normal(num_sample_points) for _ in range(num_profile)])
    x_att = np.array([rng.standard_normal(num_sample_points) for _ in range(num_attack)])
    y = rng.binomial(8, .5, num_profile)
    y_att = rng.binomial(8, .5, num_attack)

    print("Prepare traces (1/3)", end="\r")
    x9, y9, x9_att, y9_att = prepare_traces_dl(x, y, x_att, y_att)
    print("Prepare traces (2/3)", end="\r")
    (x2b, y2b), (x2b_att, y2b_att) = reduce_fixed_fixed(x9, y9), reduce_fixed_fixed(x9_att, y9_att, balanced=True)
    print("Prepare traces (3/3)", end="\r")
    (x2, y2), (x2_att, y2_att) = reduce_fixed_fixed(x9, y9), reduce_fixed_fixed(x9_att, y9_att)

    tvla_x, tvla_y = prepare_tvla(x_att, y_att)

    num_balanced = len(y2b_att)
    x9_att, y9_att = x9_att[:num_balanced], y9_att[:num_balanced]
    x2_att, y2_att = x2_att[:num_balanced], y2_att[:num_balanced]

    print("Make model (1/2)    ", end="\r")
    mdl_open = make_mlp(x9, y9, progress=False)
    print("Make model (2/2)    ", end="\r")
    mdl_wegener = make_mlp_wegener(x2b, y2b, False)

    print("TVLA                ", end="\r")
    tvla_ps = np.min(tvla_t_test(tvla_x, tvla_y, 3, progress=False), axis=1)
    rho_p = np.min(rho_test(tvla_x, tvla_y, progress=False), axis=1)[0]

    print("Predict             ", end="\r")
    dlla_k_p = dlla_known_p(mdl_open, x9_att, y9_att)
    dlla_wegener_p = wegener_p(mdl_wegener, x2b_att, y2b_att)

    print(f"TVLA t ({tvla_ps}). TVLA rho ({rho_p}). DLLA known key ({dlla_k_p}). DLLA Wegener ({dlla_wegener_p})")

    for order, p in enumerate(tvla_ps):
        if order > 0:
            store_results(f"tvla_t_{order}", p)

    store_results("tvla_rho",  rho_p)
    store_results("dlla_open", dlla_k_p)
    store_results("dlla_wegener", dlla_wegener_p)


TVLA t ([1.         0.02043338 0.00902111 0.09869055]). TVLA rho (0.016151730704236155). DLLA known key (0.7604935509063988). DLLA Wegener (0.7022022137770589)
TVLA t ([1.         0.066584   0.02892745 0.1047627 ]). TVLA rho (0.07303347113497251). DLLA known key (0.8249540521875741). DLLA Wegener (0.22451495664692928)
TVLA t ([1.         0.02218587 0.04522283 0.22542982]). TVLA rho (0.01300744965460311). DLLA known key (0.9434381047422724). DLLA Wegener (0.8829028475603927)
TVLA t ([1.         0.02501037 0.02829041 0.16348718]). TVLA rho (0.027903552181255585). DLLA known key (0.9084588988427672). DLLA Wegener (0.3558017442066054)
TVLA t ([1.         0.00702189 0.03082839 0.15912215]). TVLA rho (0.003361066250342192). DLLA known key (0.0908271216027927). DLLA Wegener (0.11028113455359195)
TVLA t ([1.         0.03523304 0.06021921 0.1985327 ]). TVLA rho (0.039331856000028045). DLLA known key (0.9356223328336075). DLLA Wegener (0.08184828859373888)
TVLA t ([1.         0.01602369 0.026329

TVLA t ([1.         0.02525439 0.00944529 0.1800988 ]). TVLA rho (0.07461414673136721). DLLA known key (0.28128254707294614). DLLA Wegener (0.241519546782394)
TVLA t ([1.         0.02651302 0.02749666 0.2554306 ]). TVLA rho (0.01828647440079998). DLLA known key (0.8582805533741164). DLLA Wegener (0.008384746265254862)
TVLA t ([1.         0.02109749 0.01966023 0.20268365]). TVLA rho (0.022681483710739688). DLLA known key (0.8927952808346988). DLLA Wegener (0.5211191920962234)
TVLA t ([1.         0.00943784 0.01550694 0.16993458]). TVLA rho (0.007563913828505738). DLLA known key (0.5535495368949014). DLLA Wegener (0.5635202873559788)
TVLA t ([1.         0.01577977 0.02163138 0.22358643]). TVLA rho (0.017833014549878533). DLLA known key (0.7003402704438136). DLLA Wegener (0.056043582888960386)
TVLA t ([1.         0.08755741 0.01403488 0.18544354]). TVLA rho (0.03548070544746123). DLLA known key (0.6242298504858146). DLLA Wegener (0.04911657447589507)
TVLA t ([1.         0.03161974 0.01993

TVLA t ([1.         0.01915625 0.01310703 0.16225982]). TVLA rho (0.048333242782350114). DLLA known key (0.3040517304213958). DLLA Wegener (0.803958934630925)
TVLA t ([1.         0.07346407 0.03352113 0.15602535]). TVLA rho (0.04776865863335698). DLLA known key (0.36113413622336366). DLLA Wegener (0.6065618201919806)
TVLA t ([1.         0.02510875 0.05289187 0.14873463]). TVLA rho (0.03340098291253279). DLLA known key (0.9469674973107617). DLLA Wegener (0.9331773115990349)
TVLA t ([1.         0.01662038 0.03232959 0.1578452 ]). TVLA rho (0.035907567835326615). DLLA known key (0.0925697833835278). DLLA Wegener (0.08006826213645857)
TVLA t ([1.         0.01175414 0.00815463 0.17890893]). TVLA rho (0.03252670944636783). DLLA known key (0.04543108367008683). DLLA Wegener (0.1688492665473652)
TVLA t ([1.         0.00633505 0.02308083 0.20699254]). TVLA rho (0.03717726915186376). DLLA known key (0.7283141222679914). DLLA Wegener (0.563881185106594)
TVLA t ([1.         0.0278471  0.00694446 0

TVLA t ([1.         0.00508795 0.026097   0.13002436]). TVLA rho (0.00410528151981423). DLLA known key (0.560339112761251). DLLA Wegener (0.45851069251300136)
TVLA t ([1.         0.02074413 0.01962431 0.11235061]). TVLA rho (0.03155566074807457). DLLA known key (0.9277859416989332). DLLA Wegener (0.07765072961356141)
TVLA t ([1.         0.02966793 0.05297032 0.14176476]). TVLA rho (0.041138754478421004). DLLA known key (0.4858626675579676). DLLA Wegener (0.0669620308806308)
TVLA t ([1.         0.04197743 0.03073391 0.16565074]). TVLA rho (0.01581638924948769). DLLA known key (0.3353470136155884). DLLA Wegener (0.8471794872893189)
TVLA t ([1.         0.02577966 0.0249005  0.24723098]). TVLA rho (0.02168122385010165). DLLA known key (0.4670335361812372). DLLA Wegener (0.9672041087251649)
TVLA t ([1.         0.03714235 0.02442954 0.14084087]). TVLA rho (0.023022421969654393). DLLA known key (0.3596415632424613). DLLA Wegener (0.6254404334997024)
TVLA t ([1.         0.04847062 0.02102164 0

TVLA t ([1.         0.01572782 0.01610504 0.20476317]). TVLA rho (0.015905392679525614). DLLA known key (0.784461049322343). DLLA Wegener (0.006815177299658922)
TVLA t ([1.         0.03669382 0.00789599 0.21271802]). TVLA rho (0.05206278192713543). DLLA known key (0.6034399893354574). DLLA Wegener (0.5210896616491582)
TVLA t ([1.         0.04986623 0.02749078 0.24544443]). TVLA rho (0.018969305633871402). DLLA known key (0.442420033018069). DLLA Wegener (0.5)
TVLA t ([1.         0.01153952 0.01476891 0.12394917]). TVLA rho (0.027379424450884926). DLLA known key (0.9995641493092098). DLLA Wegener (0.9141136112021442)
TVLA t ([1.         0.03470476 0.03047384 0.1096443 ]). TVLA rho (0.013360311935641292). DLLA known key (0.8571639163921183). DLLA Wegener (0.21618695127160653)
TVLA t ([1.         0.07555631 0.02258978 0.19377483]). TVLA rho (0.03745813210129798). DLLA known key (0.3711298168276297). DLLA Wegener (0.1254036654675741)
TVLA t ([1.         0.02413905 0.02852037 0.17049557]). 

...

TVLA t ([1.         0.0461404  0.05894142 0.10788471]). TVLA rho (0.0066809113128591105). DLLA known key (0.27485878319325874). DLLA Wegener (0.8292814472789918)
TVLA t ([1.         0.0501167  0.00688755 0.20428811]). TVLA rho (0.018987057378047883). DLLA known key (0.6432889094100718). DLLA Wegener (0.22841052893598446)
TVLA t ([1.         0.02496505 0.023757   0.15818894]). TVLA rho (0.016839093467497096). DLLA known key (0.5300995476812674). DLLA Wegener (0.043702187409219495)
TVLA t ([1.         0.00678593 0.02362036 0.13644091]). TVLA rho (0.02572759331982187). DLLA known key (0.7219420711591256). DLLA Wegener (0.43674640757167604)
TVLA t ([1.         0.06321516 0.02625412 0.18035727]). TVLA rho (0.03961420776648342). DLLA known key (0.4511611978453707). DLLA Wegener (0.6620873791562027)
TVLA t ([1.         0.02271951 0.02587244 0.21769583]). TVLA rho (0.051382099736701874). DLLA known key (0.6285984506513675). DLLA Wegener (0.2789760611697357)
TVLA t ([1.         0.0141318  0.014

TVLA t ([1.         0.02527266 0.03585602 0.1415419 ]). TVLA rho (0.034790309215559596). DLLA known key (0.8808706220535754). DLLA Wegener (0.5)
TVLA t ([1.         0.02846346 0.05924189 0.20616414]). TVLA rho (0.02936372022477579). DLLA known key (0.7340795487328216). DLLA Wegener (0.8396805731828136)
TVLA t ([1.         0.01793108 0.0314302  0.17535139]). TVLA rho (0.017019740654198458). DLLA known key (0.8573436940635026). DLLA Wegener (0.5420035351670658)
TVLA t ([1.         0.03511519 0.02228723 0.18169879]). TVLA rho (0.04779014661347139). DLLA known key (0.3404172480763209). DLLA Wegener (0.19920963786176077)
TVLA t ([1.         0.02669803 0.02265144 0.09915746]). TVLA rho (0.01713982769198265). DLLA known key (0.8314111096541943). DLLA Wegener (0.2815823833675769)
TVLA t ([1.         0.01276581 0.0488429  0.2069806 ]). TVLA rho (0.03650342136728479). DLLA known key (0.6009045097492995). DLLA Wegener (0.4574020507055861)
TVLA t ([1.         0.03376715 0.01977817 0.16628306]). TV

TVLA t ([1.         0.03187614 0.02845532 0.03263784]). TVLA rho (0.02788858307220441). DLLA known key (0.5316490129886113). DLLA Wegener (0.6812087429885127)
TVLA t ([1.         0.01237155 0.05200897 0.19248698]). TVLA rho (0.008575659066702995). DLLA known key (0.8009776415678033). DLLA Wegener (0.8776965745860352)
TVLA t ([1.         0.05058099 0.04461899 0.24374021]). TVLA rho (0.04379287165265438). DLLA known key (0.174987027290265). DLLA Wegener (0.9232092370982791)
TVLA t ([1.         0.04419603 0.01099631 0.15592553]). TVLA rho (0.06146002815221619). DLLA known key (0.2673856053502561). DLLA Wegener (0.4127954927480786)
TVLA t ([1.         0.03351408 0.01486983 0.26778528]). TVLA rho (0.030143894348733716). DLLA known key (0.5042025275216429). DLLA Wegener (0.8542473901569916)
TVLA t ([1.         0.01224302 0.0242209  0.2019543 ]). TVLA rho (0.021983630984665714). DLLA known key (0.42446955138890197). DLLA Wegener (0.27994704257739356)
TVLA t ([1.         0.02411152 0.03211797 

TVLA t ([1.         0.00911851 0.01914142 0.0765808 ]). TVLA rho (0.024019643495513884). DLLA known key (0.05866763194312484). DLLA Wegener (0.7731927736823085)
TVLA t ([1.         0.04096532 0.01926423 0.21650443]). TVLA rho (0.026626722070992804). DLLA known key (0.045958652632788026). DLLA Wegener (0.0625432376641161)
TVLA t ([1.         0.06674735 0.02855583 0.14556791]). TVLA rho (0.043320069481197966). DLLA known key (0.5401315585564426). DLLA Wegener (0.9957048376064538)
TVLA t ([1.         0.04746282 0.05898934 0.20412941]). TVLA rho (0.024972605079050865). DLLA known key (0.7151308993665275). DLLA Wegener (0.9785133723148935)
TVLA t ([1.         0.02090838 0.05104037 0.22181467]). TVLA rho (0.0382000647329005). DLLA known key (0.44954858940966713). DLLA Wegener (0.8864800580144339)
TVLA t ([1.         0.0190186  0.02133088 0.16827094]). TVLA rho (0.04046816548091628). DLLA known key (0.28287939040644894). DLLA Wegener (0.16953250667891817)
TVLA t ([1.         0.04236504 0.0419

TVLA t ([1.         0.01581993 0.01702613 0.176511  ]). TVLA rho (0.018923820737221316). DLLA known key (0.9224675363222672). DLLA Wegener (0.08514494655449012)
TVLA t ([1.         0.01904591 0.01982261 0.18418302]). TVLA rho (0.025394322656709767). DLLA known key (0.8526904878976655). DLLA Wegener (0.5635202873559788)
TVLA t ([1.         0.03466299 0.04298099 0.21506738]). TVLA rho (0.021925168224415842). DLLA known key (0.6662578735208351). DLLA Wegener (0.736312044962153)
TVLA t ([1.         0.00349635 0.01739344 0.21725896]). TVLA rho (0.049111393292295). DLLA known key (0.9432978143442105). DLLA Wegener (0.7389321948674445)
TVLA t ([1.         0.02651959 0.02141133 0.13408103]). TVLA rho (0.025334969445766357). DLLA known key (0.5979979530327911). DLLA Wegener (0.39586150040960355)
TVLA t ([1.         0.03376191 0.01696759 0.14698067]). TVLA rho (0.007644267032018116). DLLA known key (0.2782272349975038). DLLA Wegener (0.8316658161949806)
TVLA t ([1.         0.03614068 0.04461895 